In [ ]:
#@title Download dataset
import kagglehub

# Download latest version
path = kagglehub.dataset_download("fabianavinci/guitar-chords-v2")

print("Path to dataset files:", path)

100%|██████████| 729M/729M [00:08<00:00, 85.2MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/fabianavinci/guitar-chords-v2/versions/3


In [ ]:
#@title Move dataset to content folder
import shutil
import os

SRC = path
DST = "/content/guitar-chords-v2"

# nếu đã tồn tại thì xoá
if os.path.exists(DST):
    shutil.rmtree(DST)

shutil.copytree(SRC, DST)

print("Copied to:", DST)
print(os.listdir(DST))

Copied to: /content/guitar-chords-v2
['Training', 'Test']


In [ ]:
#@title Import libraries
import os
import glob
import numpy as np
import librosa

from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.utils.class_weight import compute_class_weight

BASE_DIR = "/content/guitar-chords-v2"
TRAIN_DIR = os.path.join(BASE_DIR, "Training")
TEST_DIR  = os.path.join(BASE_DIR, "Test")

In [ ]:
#@title Load files
def load_split(split_dir):
    paths = []
    labels = []

    for chord in sorted(os.listdir(split_dir)):
        chord_dir = os.path.join(split_dir, chord)
        if not os.path.isdir(chord_dir):
            continue

        for wav in glob.glob(os.path.join(chord_dir, "*.wav")):
            paths.append(wav)
            labels.append(chord)

    return paths, np.array(labels)

train_paths, train_chords = load_split(TRAIN_DIR)
val_paths,   val_chords   = load_split(TEST_DIR)

print("Train samples:", len(train_paths))
print("Val samples:", len(val_paths))
print("Chord classes:", sorted(set(train_chords)))

Train samples: 1440
Val samples: 320
Chord classes: [np.str_('Am'), np.str_('Bb'), np.str_('Bdim'), np.str_('C'), np.str_('Dm'), np.str_('Em'), np.str_('F'), np.str_('G')]


In [ ]:
#@title Feature extraction (log mel)
def extract_feature(path, sr=22050, n_mels=64):
    y, sr = librosa.load(path, sr=sr, mono=True)

    mel = librosa.feature.melspectrogram(
        y=y, sr=sr,
        n_mels=n_mels,
        fmin=80, fmax=2000
    )

    logmel = np.log(mel + 1e-6)

    # pool theo frequency
    feat = logmel.mean(axis=1)              # (64,)
    feat = feat.reshape(16, 4).mean(axis=1) # (16,)

    return feat

In [ ]:
#@title Feature matrices
from tqdm import tqdm
import numpy as np

X_train = np.stack([
    extract_feature(p)
    for p in tqdm(train_paths, desc="Extract train features")
])

X_val = np.stack([
    extract_feature(p)
    for p in tqdm(val_paths, desc="Extract val features")
])

print("Feature shape:", X_train.shape)

Extract val features: 100%|██████████| 320/320 [00:07<00:00, 44.59it/s]

Feature shape: (1440, 16)


In [ ]:
#@title Encode label
le_chord = LabelEncoder()

y_train = le_chord.fit_transform(train_chords)
y_val   = le_chord.transform(val_chords)

print("Encoded classes:", le_chord.classes_)

Encoded classes: ['Am' 'Bb' 'Bdim' 'C' 'Dm' 'Em' 'F' 'G']


In [ ]:
#@title Scale feature
scaler = StandardScaler()
X_train_s = scaler.fit_transform(X_train)
X_val_s   = scaler.transform(X_val)

scaler_mean  = scaler.mean_
scaler_scale = scaler.scale_

In [ ]:
#@title Export mean and scale to C array
def to_c_array(name, arr, per_line=8):
    print(f"static const float {name}[{len(arr)}] = {{")
    for i in range(0, len(arr), per_line):
        chunk = ", ".join(f"{x:.8f}f" for x in arr[i:i+per_line])
        print("  " + chunk + ("," if i + per_line < len(arr) else ""))
    print("};\n")

to_c_array("CHORD_MEAN", scaler.mean_)
to_c_array("CHORD_SCALE", scaler.scale_)

static const float CHORD_MEAN[16] = {
  -2.03650455f, -2.26095502f, -2.45058618f, -4.14766748f, -4.66261261f, -4.88741472f, -5.50021994f, -5.58704452f,
  -6.32571979f, -6.25388386f, -6.89863926f, -7.02966055f, -6.83384801f, -6.77439976f, -6.81123306f, -7.58411825f
};

static const float CHORD_SCALE[16] = {
  2.53589825f, 1.97801876f, 2.00723825f, 2.00208736f, 2.12831373f, 2.20478826f, 2.28146183f, 2.42331475f,
  2.13611270f, 2.04229769f, 2.11201004f, 2.17831763f, 2.17883348f, 2.29924031f, 2.48667369f, 2.30689137f
};



In [ ]:
#@title Feature shape
print("X_train shape:", X_train_s.shape)
print("X_val shape:", X_val_s.shape)
print("X_train type:", type(X_train_s), "dtype:", X_train_s.dtype)
print("X_val type:", type(X_val_s), "dtype:", X_val_s.dtype)

X_train shape: (1440, 16)
X_val shape: (320, 16)
X_train type: <class 'numpy.ndarray'> dtype: float32
X_val type: <class 'numpy.ndarray'> dtype: float32


In [ ]:
#@title Compute class weight
class_weight = dict(zip(
    np.unique(y_train),
    compute_class_weight(
        class_weight="balanced",
        classes=np.unique(y_train),
        y=y_train
    )
))

print("Class weight:", class_weight)

Class weight: {np.int64(0): np.float64(1.0), np.int64(1): np.float64(1.0), np.int64(2): np.float64(1.0), np.int64(3): np.float64(1.0), np.int64(4): np.float64(1.0), np.int64(5): np.float64(1.0), np.int64(6): np.float64(1.0), np.int64(7): np.float64(1.0)}


In [ ]:
#@title Training + Finetuning Logistic Regression
C_list = [0.01, 0.1, 1.0, 10.0]

best_acc = -1
best_model = None
best_pred = None
best_C = None

for C in C_list:
    model = LogisticRegression(
        C=C,
        max_iter=1000,
        class_weight=class_weight,
        solver="lbfgs"
    )

    model.fit(X_train_s, y_train)
    pred = model.predict(X_val_s)

    acc = accuracy_score(y_val, pred)
    print(f"C={C} | chord acc={acc:.4f}")

    if acc > best_acc:
        best_acc = acc
        best_model = model
        best_pred = pred
        best_C = C

print("\n=== BEST MODEL ===")
print("Best C:", best_C)
print("Best chord ACC:", best_acc)

C=0.01 | chord acc=0.5469
C=0.1 | chord acc=0.5906
C=1.0 | chord acc=0.6562
C=10.0 | chord acc=0.6594

=== BEST MODEL ===
Best C: 10.0
Best chord ACC: 0.659375


In [ ]:
#@title Training + Finetuning Linear SVM
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

C_values = [0.01, 0.1, 1.0, 10.0]
best_acc = -1

for C in C_values:
    model = LinearSVC(
        C=C,
        class_weight="balanced",
        max_iter=5000
    )
    model.fit(X_train_s, y_train)
    pred = model.predict(X_val_s)

    acc = accuracy_score(y_val, pred)
    print(f"C={C} | acc={acc:.4f}")

    if acc > best_acc:
        best_acc = acc
        best_model = model

C=0.01 | acc=0.6000
C=0.1 | acc=0.6094
C=1.0 | acc=0.6094
C=10.0 | acc=0.6094


In [ ]:
#@title Training + Finetuning RidgeClassifier
from sklearn.linear_model import RidgeClassifier

alphas = [0.1, 1.0, 10.0, 100.0]

for a in alphas:
    model = RidgeClassifier(alpha=a, class_weight="balanced")
    model.fit(X_train_s, y_train)
    pred = model.predict(X_val_s)
    print(f"alpha={a} | acc={accuracy_score(y_val, pred):.4f}")

alpha=0.1 | acc=0.5594
alpha=1.0 | acc=0.5594
alpha=10.0 | acc=0.5656
alpha=100.0 | acc=0.5781


In [ ]:
#@title Training Polymonial Logistic
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

model = Pipeline([
    ("poly", PolynomialFeatures(degree=2, include_bias=False)),
    ("clf", LogisticRegression(
        C=1.0,
        max_iter=3000,
        class_weight="balanced"
    ))
])

model.fit(X_train_s, y_train)
pred = model.predict(X_val_s)
print("ACC:", accuracy_score(y_val, pred))

ACC: 0.6


In [ ]:
#@title Training + Finetuning MLP
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

# Danh sách các cấu hình hidden layers để thử
configs = [
    (16,),       # 1 layer nhỏ
    (32,),
    (64, 32),    # 2 layer
    (128, 64),
]

best_acc = -1
best_config = None
best_model = None

for cfg in configs:
    print(f"\n--- Training MLP with hidden_layer_sizes={cfg} ---")
    mlp = MLPClassifier(
        hidden_layer_sizes=cfg,
        activation="relu",
        solver="adam",
        max_iter=500,
        early_stopping=True,
        n_iter_no_change=10,
        random_state=42
    )
    mlp.fit(X_train_s, y_train)
    pred = mlp.predict(X_val_s)
    acc = accuracy_score(y_val, pred)
    print("Validation ACC:", acc)

    if acc > best_acc:
        best_acc = acc
        best_config = cfg
        best_model = mlp

print("\n=== BEST CONFIG ===")
print("hidden_layer_sizes:", best_config)
print("Best Validation ACC:", best_acc)


--- Training MLP with hidden_layer_sizes=(16,) ---
Validation ACC: 0.6

--- Training MLP with hidden_layer_sizes=(32,) ---
Validation ACC: 0.5875

--- Training MLP with hidden_layer_sizes=(64, 32) ---
Validation ACC: 0.596875

--- Training MLP with hidden_layer_sizes=(128, 64) ---
Validation ACC: 0.671875

=== BEST CONFIG ===
hidden_layer_sizes: (128, 64)
Best Validation ACC: 0.671875


In [ ]:
#@title Grid search LR + MLP
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

# --- Tạo polynomial features nhẹ ---
poly = PolynomialFeatures(degree=2, include_bias=False)
X_train_poly = poly.fit_transform(X_train_s)
X_val_poly   = poly.transform(X_val_s)
print("Polynomial feature shape:", X_train_poly.shape)

# --- Grid search Logistic Regression ---
C_values = [1.0, 10.0, 50.0]
best_lr_acc = -1
best_lr_model = None
best_lr_C = None

for C in C_values:
    lr = LogisticRegression(
        C=C,
        max_iter=500,
        solver='lbfgs',
        class_weight='balanced',
        n_jobs=-1,
        random_state=42
    )
    lr.fit(X_train_poly, y_train)
    pred = lr.predict(X_val_poly)
    acc = accuracy_score(y_val, pred)
    print(f"LR | C={C:>5} | Validation ACC={acc:.4f}")
    if acc > best_lr_acc:
        best_lr_acc = acc
        best_lr_model = lr
        best_lr_C = C

print(f"\n=== BEST LOGISTIC REGRESSION ===")
print(f"C={best_lr_C} | ACC={best_lr_acc:.4f}")

# --- Grid search MLP 2 lớp ---
mlp_configs = [
    (64, 32),
    (128, 64),
    (64, 64)
]
activations = ['relu', 'tanh']

best_mlp_acc = -1
best_mlp_model = None
best_mlp_cfg = None
best_mlp_act = None

for cfg in mlp_configs:
    for act in activations:
        mlp = MLPClassifier(
            hidden_layer_sizes=cfg,
            activation=act,
            solver='adam',
            max_iter=500,
            early_stopping=True,
            n_iter_no_change=10,
            alpha=1e-4,
            random_state=42
        )
        mlp.fit(X_train_s, y_train)
        pred = mlp.predict(X_val_s)
        acc = accuracy_score(y_val, pred)
        print(f"MLP | cfg={cfg} | act={act:>4} | ACC={acc:.4f}")
        if acc > best_mlp_acc:
            best_mlp_acc = acc
            best_mlp_model = mlp
            best_mlp_cfg = cfg
            best_mlp_act = act

print(f"\n=== BEST MLP ===")
print(f"hidden_layer_sizes={best_mlp_cfg} | activation={best_mlp_act} | ACC={best_mlp_acc:.4f}")

Polynomial feature shape: (1440, 152)
LR | C=  1.0 | Validation ACC=0.6031
LR | C= 10.0 | Validation ACC=0.5437
LR | C= 50.0 | Validation ACC=0.5344

=== BEST LOGISTIC REGRESSION ===
C=1.0 | ACC=0.6031
MLP | cfg=(64, 32) | act=relu | ACC=0.6062
MLP | cfg=(64, 32) | act=tanh | ACC=0.6219
MLP | cfg=(128, 64) | act=relu | ACC=0.6750
MLP | cfg=(128, 64) | act=tanh | ACC=0.6531
MLP | cfg=(64, 64) | act=relu | ACC=0.6250
MLP | cfg=(64, 64) | act=tanh | ACC=0.6438

=== BEST MLP ===
hidden_layer_sizes=(128, 64) | activation=relu | ACC=0.6750


In [ ]:
#@title Feature engineering
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report

# --- Feature engineering ---
def mel_to_chroma(X_mel):
    """
    Chroma-like features từ log-mel (n_samples, 16) -> (n_samples, 12)
    """
    n_samples, n_bins = X_mel.shape
    chroma = np.zeros((n_samples, 12))
    for i in range(12):
        bins_idx = [i, (i+12)%n_bins] if n_bins>12 else [i]
        chroma[:, i] = X_mel[:, bins_idx].mean(axis=1)
    chroma = chroma / (chroma.sum(axis=1, keepdims=True) + 1e-6)
    return chroma

# Delta features
def delta_features(X):
    return np.diff(X, axis=1, prepend=0)

# Combine features
X_train_feat = np.concatenate([
    X_train_s,                      # original 16-dim
    mel_to_chroma(X_train_s),       # 12-dim chroma-like
    delta_features(X_train_s)       # 16-dim delta
], axis=1)

X_val_feat = np.concatenate([
    X_val_s,
    mel_to_chroma(X_val_s),
    delta_features(X_val_s)
], axis=1)

print("Final feature shape:", X_train_feat.shape)

# --- Standard scaling ---
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_feat)
X_val_scaled   = scaler.transform(X_val_feat)

# --- Logistic Regression C=10 ---
lr = LogisticRegression(
    C=10.0,
    max_iter=500,
    solver='lbfgs',
    class_weight='balanced',
    n_jobs=-1,
    random_state=42
)
lr.fit(X_train_scaled, y_train)
pred_lr = lr.predict(X_val_scaled)
acc_lr = accuracy_score(y_val, pred_lr)
print(f"\nLOGISTIC REGRESSION ACC: {acc_lr:.4f}")
print(classification_report(y_val, pred_lr))

# --- MLP (128,64, tanh) ---
mlp = MLPClassifier(
    hidden_layer_sizes=(128, 64),
    activation='tanh',
    solver='adam',
    max_iter=500,
    early_stopping=True,
    n_iter_no_change=10,
    random_state=42
)
mlp.fit(X_train_scaled, y_train)
pred_mlp = mlp.predict(X_val_scaled)
acc_mlp = accuracy_score(y_val, pred_mlp)
print(f"\nMLP (128,64,tanh) ACC: {acc_mlp:.4f}")
print(classification_report(y_val, pred_mlp))

Final feature shape: (1440, 44)

LOGISTIC REGRESSION ACC: 0.6531
              precision    recall  f1-score   support

           0       0.74      0.50      0.60        40
           1       0.80      0.93      0.86        40
           2       0.58      0.47      0.52        40
           3       0.60      0.88      0.71        40
           4       0.73      0.90      0.81        40
           5       0.58      0.72      0.64        40
           6       0.53      0.47      0.50        40
           7       0.67      0.35      0.46        40

    accuracy                           0.65       320
   macro avg       0.65      0.65      0.64       320
weighted avg       0.65      0.65      0.64       320


MLP (128,64,tanh) ACC: 0.6719
              precision    recall  f1-score   support

           0       0.72      0.45      0.55        40
           1       0.80      0.88      0.83        40
           2       0.49      0.47      0.48        40
           3       0.65      0.88   

In [ ]:
#@title Feature engineering (Polynomial feature)
import numpy as np
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report

# --- Feature engineering ---
def mel_to_chroma(X_mel):
    n_samples, n_bins = X_mel.shape
    chroma = np.zeros((n_samples, 12))
    for i in range(12):
        bins_idx = [i, (i+12)%n_bins] if n_bins>12 else [i]
        chroma[:, i] = X_mel[:, bins_idx].mean(axis=1)
    chroma = chroma / (chroma.sum(axis=1, keepdims=True) + 1e-6)
    return chroma

def delta_features(X):
    return np.diff(X, axis=1, prepend=0)

# Combine features: original + chroma + delta
X_train_feat = np.concatenate([
    X_train_s,
    mel_to_chroma(X_train_s),
    delta_features(X_train_s)
], axis=1)
X_val_feat = np.concatenate([
    X_val_s,
    mel_to_chroma(X_val_s),
    delta_features(X_val_s)
], axis=1)

print("Before polynomial features:", X_train_feat.shape)

# --- Polynomial features degree 2 ---
poly = PolynomialFeatures(degree=2, include_bias=False)
X_train_poly = poly.fit_transform(X_train_feat)
X_val_poly   = poly.transform(X_val_feat)

print("After polynomial features:", X_train_poly.shape)

# --- Standard scaling ---
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_poly)
X_val_scaled   = scaler.transform(X_val_poly)

# --- Logistic Regression C=10 ---
lr = LogisticRegression(
    C=10.0,
    max_iter=500,
    solver='lbfgs',
    class_weight='balanced',
    n_jobs=-1,
    random_state=42
)
lr.fit(X_train_scaled, y_train)
pred_lr = lr.predict(X_val_scaled)
acc_lr = accuracy_score(y_val, pred_lr)
print(f"\nLOGISTIC REGRESSION ACC: {acc_lr:.4f}")
print(classification_report(y_val, pred_lr))

# --- MLP (128,64, tanh) ---
mlp = MLPClassifier(
    hidden_layer_sizes=(128, 64),
    activation='tanh',
    solver='adam',
    max_iter=500,
    early_stopping=True,
    n_iter_no_change=10,
    random_state=42
)
mlp.fit(X_train_scaled, y_train)
pred_mlp = mlp.predict(X_val_scaled)
acc_mlp = accuracy_score(y_val, pred_mlp)
print(f"\nMLP (128,64,tanh) ACC: {acc_mlp:.4f}")
print(classification_report(y_val, pred_mlp))

Before polynomial features: (1440, 44)
After polynomial features: (1440, 1034)

LOGISTIC REGRESSION ACC: 0.6031
              precision    recall  f1-score   support

           0       0.54      0.50      0.52        40
           1       0.64      0.57      0.61        40
           2       0.49      0.70      0.58        40
           3       0.65      0.50      0.56        40
           4       0.87      1.00      0.93        40
           5       0.73      0.60      0.66        40
           6       0.37      0.45      0.40        40
           7       0.65      0.50      0.56        40

    accuracy                           0.60       320
   macro avg       0.62      0.60      0.60       320
weighted avg       0.62      0.60      0.60       320


MLP (128,64,tanh) ACC: 0.6344
              precision    recall  f1-score   support

           0       0.60      0.53      0.56        40
           1       0.68      0.62      0.65        40
           2       0.58      0.65      0.61

In [ ]:
#@title Kernel approximation for MLP
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.kernel_approximation import RBFSampler
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# --- Feature scaling ---
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_s)
X_val_scaled   = scaler.transform(X_val_s)

# --- Define configurations to try ---
mlp_configs = [
    {"layers": (128, 64), "activation": "tanh", "use_rbf": False},
    {"layers": (128, 64), "activation": "tanh", "use_rbf": True},
    {"layers": (256, 128, 64), "activation": "tanh", "use_rbf": False},
    {"layers": (256, 128, 64), "activation": "tanh", "use_rbf": True},
    {"layers": (128, 64), "activation": "relu", "use_rbf": False},
    {"layers": (128, 64), "activation": "relu", "use_rbf": True},
    {"layers": (256, 128, 64), "activation": "relu", "use_rbf": False},
    {"layers": (256, 128, 64), "activation": "relu", "use_rbf": True},
]

best_acc = -1
best_config = None
best_model = None

for cfg in mlp_configs:
    print("\n=== TRAINING CONFIG ===")
    print(cfg)

    # Prepare features
    if cfg["use_rbf"]:
        rbf_feature = RBFSampler(gamma=0.5, n_components=128, random_state=42)
        X_train_feat = rbf_feature.fit_transform(X_train_scaled)
        X_val_feat   = rbf_feature.transform(X_val_scaled)
    else:
        X_train_feat = X_train_scaled
        X_val_feat   = X_val_scaled

    # Train MLP
    mlp = MLPClassifier(
        hidden_layer_sizes=cfg["layers"],
        activation=cfg["activation"],
        solver="adam",
        max_iter=500,
        early_stopping=True,
        n_iter_no_change=10,
        random_state=42
    )
    mlp.fit(X_train_feat, y_train)
    pred = mlp.predict(X_val_feat)
    acc = accuracy_score(y_val, pred)

    print("Validation ACC:", acc)
    print(classification_report(y_val, pred))

    if acc > best_acc:
        best_acc = acc
        best_config = cfg
        best_model = mlp

print("\n=== BEST MLP CONFIG ===")
print("Config:", best_config)
print("Validation ACC:", best_acc)


=== TRAINING CONFIG ===
{'layers': (128, 64), 'activation': 'tanh', 'use_rbf': False}
Validation ACC: 0.653125
              precision    recall  f1-score   support

           0       0.69      0.50      0.58        40
           1       0.80      0.90      0.85        40
           2       0.36      0.40      0.38        40
           3       0.67      0.88      0.76        40
           4       0.78      1.00      0.88        40
           5       0.77      0.60      0.68        40
           6       0.49      0.45      0.47        40
           7       0.65      0.50      0.56        40

    accuracy                           0.65       320
   macro avg       0.65      0.65      0.64       320
weighted avg       0.65      0.65      0.64       320


=== TRAINING CONFIG ===
{'layers': (128, 64), 'activation': 'tanh', 'use_rbf': True}
Validation ACC: 0.203125
              precision    recall  f1-score   support

           0       0.10      0.12      0.11        40
           1     

In [ ]:
#@title Export best model weight to .h
import numpy as np

HEADER_FILE = "mlp_weights.h"
PREFIX = "MLP"

# Input scale
x_max = np.percentile(np.abs(X_train_scaled), 99.9)
INPUT_SCALE = 127.0 / x_max

mlp_best = best_model

coefs = mlp_best.coefs_            # list of (in_dim, out_dim)
intercepts = mlp_best.intercepts_  # list of (out_dim,)

with open(HEADER_FILE, "w") as f:
    f.write("#pragma once\n\n")
    f.write("#include <stdint.h>\n\n")

    f.write("// ===== Quantization config =====\n")
    f.write(f"#define {PREFIX}_INPUT_SCALE {INPUT_SCALE:.8f}f\n\n")

    prev_scale = INPUT_SCALE

    for layer_idx, (W, b) in enumerate(zip(coefs, intercepts)):
        # sklearn: W shape = (in_dim, out_dim)
        in_dim, out_dim = W.shape

        # transpose cho C: (out_dim, in_dim)
        W = W.T

        # ---- Quantize weights ----
        W_max = np.max(np.abs(W))
        W_scale = 127.0 / W_max if W_max != 0 else 1.0
        W_q = np.round(W * W_scale).astype(np.int8)

        # ---- Quantize bias (INT32, đúng chuẩn) ----
        b_scale = prev_scale * W_scale
        b_q = np.round(b * b_scale).astype(np.int32)

        # ---- Write metadata ----
        f.write(f"// ===== Layer {layer_idx} =====\n")
        f.write(f"#define {PREFIX}_L{layer_idx}_IN  {in_dim}\n")
        f.write(f"#define {PREFIX}_L{layer_idx}_OUT {out_dim}\n")
        f.write(f"#define {PREFIX}_L{layer_idx}_W_SCALE {W_scale:.8f}f\n")
        f.write(f"#define {PREFIX}_L{layer_idx}_B_SCALE {b_scale:.8f}f\n\n")

        # ---- Write weights ----
        f.write(f"const int8_t {PREFIX}_W{layer_idx}[{out_dim}][{in_dim}] = {{\n")
        for row in W_q:
            f.write("  { " + ", ".join(map(str, row)) + " },\n")
        f.write("};\n\n")

        # ---- Write bias ----
        f.write(f"const int32_t {PREFIX}_B{layer_idx}[{out_dim}] = {{\n")
        f.write("  " + ", ".join(map(str, b_q)) + "\n")
        f.write("};\n\n")

        prev_scale = b_scale

print(f"Exported quantized MLP weights to {HEADER_FILE}")

Exported quantized MLP weights to mlp_weights.h


In [ ]:
# Save the best model and scaler
import joblib
joblib.dump(best_model, "chord_mlp_model.pkl")
joblib.dump(scaler, "chord_scaler.pkl")
print("Saved chord model to chord_mlp_model.pkl and scaler to chord_scaler.pkl")